<a href="https://colab.research.google.com/github/Ali-Alameer/Deep-Learning/blob/main/week7_object_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train YOLOv7 on a Custom Dataset

This workshop is based on the [YOLOv7 repository](https://github.com/WongKinYiu/yolov7) by WongKinYiu.

### **Steps Covered in this Tutorial**

To train our detector we take the following steps:

* Install YOLOv7 dependencies
* Load custom dataset from Roboflow in YOLOv7 format
* Run YOLOv7 training
* Evaluate YOLOv7 performance
* Run YOLOv7 inference on test images
* OPTIONAL: Deployment
* OPTIONAL: Active Learning


### Preparing a Custom Dataset

In this workshop, we will utilise a previosly collected dataset with the aim to detect pig heads and rears to then quantify interactions [Head and Rear Detection](https://salford.figshare.com/articles/dataset/Automated_detection_and_quantification_of_contact_behaviour_in_pigs_using_deep_learning/21346767).

If you already have your own images (and, optionally, annotations), you can convert your dataset using [Roboflow](https://roboflow.com), a set of tools developers use to build better computer vision models quickly and accurately. 1

#Install Dependencies

_(Remember to choose GPU in Runtime if not already selected. Runtime --> Change Runtime Type --> Hardware accelerator --> GPU)_

In [ ]:
!nvidia-smi

In [ ]:
# Download YOLOv7 repository and install requirements

# !git clone https://github.com/WongKinYiu/yolov7
# %cd yolov7
# !pip install -r requirements.txt

# current version of YOLOv7 is not compatible with pytorch>1.12.1 and numpy>1.20.1
# until the appropriate changes get made to the main repository, we will be using a fork containing the patched code
# you can track the progress here: https://github.com/roboflow/notebooks/issues/27
!git clone https://github.com/SkalskiP/yolov7.git
%cd yolov7
!git checkout fix/problems_associated_with_the_latest_versions_of_pytorch_and_numpy
!pip install -r requirements.txt


- Download the dataset from the provided link: [Automated Detection and Quantification of Contact Behaviour in Pigs Dataset](https://salford.figshare.com/articles/dataset/Automated_detection_and_quantification_of_contact_behaviour_in_pigs_using_deep_learning/21346767)
- Extract the downloaded dataset and locate the folder that contains the pig parts detection with any type of annotation. This folder will be used for further processing.
- Create an account with Roboflow if you don't have one already. Roboflow is a platform that helps in preparing and managing datasets for computer vision tasks.
- Once logged in to Roboflow, create a new object detection project by selecting the appropriate option from the project creation menu.
- Follow the instructions to set up the project and select the folder containing the dataset that you extracted in step 2. Upload the dataset to Roboflow and wait for the upload process to complete.
- After the dataset is uploaded, you can choose to split the dataset into training, validating and testing sets (code will not work if not all three are available) based on your requirements. Follow the provided options and instructions to perform the split and wait for the process to complete.
- Once the dataset is split, you will be presented with several steps. Click "Continue" on each step without applying any preprocessing or augmentation. These steps are optional and can be used to preprocess and augment the dataset if required. For this exercise, we will skip them.
- Finally, click "Generate Dataset" to proceed. On the following page, you will find an option to export the dataset. Choose the export format suitable for YOLOv7 and select "Show Download Code" to obtain the code needed for downloading the dataset.
- Copy the provided code, which may include commands or scripts, and use it to download the exported dataset. Ensure that you have the necessary dependencies, such as PyTorch, installed to run YOLOv7 on the downloaded dataset.
By following these steps, students can obtain an object detection dataset in the appropriate format for running YOLOv7 using the Roboflow platform and the provided dataset.

# Download Correctly Formatted Custom Data

Next, we'll download our dataset in the right format. Use the `YOLOv7 PyTorch` export. Note that this model requires YOLO TXT annotations, a custom YAML file, and organized directories. The roboflow export writes this for us and saves it in the correct spot.


In [ ]:
# REPLACE with your custom code snippet generated above

!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="YOUR API KEY")
project = rf.workspace("YOUR-WORKSPACE").project("YOUR-PROJECT")
dataset = project.version(3).download("yolov7")

# Begin Custom Training

We're ready to start custom training.

NOTE: We will only modify one of the YOLOv7 training defaults in our example: `epochs`. We will adjust from 300 to 100 epochs in our example for speed.

In [ ]:
# download COCO starting checkpoint
%cd /content/yolov7
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7_training.pt

In [ ]:
# run this cell to begin training; for real implemenation increase number of epochs
%cd /content/yolov7
!python train.py --batch 16 --epochs 1 --data {dataset.location}/data.yaml --weights 'yolov7_training.pt' --device 0


# Evaluation

We can evaluate the performance of our custom training using the provided evalution script.

In [ ]:
# Run evaluation
!python detect.py --weights runs/train/exp/weights/best.pt --conf 0.1 --source {dataset.location}/test/images


In [ ]:
#display inference on ALL test images

import glob
from IPython.display import Image, display

i = 0
limit = 10000 # max images to print
for imageName in glob.glob('/content/yolov7/runs/detect/exp/*.jpg'): #assuming JPG
    if i < limit:
      display(Image(filename=imageName))
      print("\n")
    i = i + 1


# OPTIONAL: Deployment

To deploy, you'll need to export your weights and save them to use later.

In [ ]:
# optional, zip to download weights and results locally

!zip -r export.zip runs/detect
!zip -r export.zip runs/train/exp/weights/best.pt
!zip export.zip runs/train/exp/*

# OPTIONAL: Active Learning Example

Once our first training run is complete, we should use our model to help identify which images are most problematic in order to investigate, annotate, and improve our dataset (and, therefore, model).

To do that, we can execute code that automatically uploads images back to our hosted dataset if the image is a specific class or below a given confidence threshold.


In [ ]:
# # setup access to your workspace
# rf = Roboflow(api_key="YOUR_API_KEY")                               # used above to load data
# inference_project =  rf.workspace().project("YOUR_PROJECT_NAME")    # used above to load data
# model = inference_project.version(1).model

# upload_project = rf.workspace().project("YOUR_PROJECT_NAME")

# print("inference reference point: ", inference_project)
# print("upload destination: ", upload_project)

In [ ]:
# # example upload: if prediction is below a given confidence threshold, upload it

# confidence_interval = [10,70]                                   # [lower_bound_percent, upper_bound_percent]

# for prediction in predictions:                                  # predictions list to loop through
#   if(prediction['confidence'] * 100 >= confidence_interval[0] and
#           prediction['confidence'] * 100 <= confidence_interval[1]):

#           # upload on success!
#           print(' >> image uploaded!')
#           upload_project.upload(image, num_retry_uploads=3)     # upload image in question